In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import os

from pprint import pprint

from pricing.models.abtests import *
from pricing.utils import convert_numeric
# from src.utils import *
from pprint import pprint
import os


from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

from pricing.operations.experiments import level_1_kind_1_courses_1

print("Using as base_host: ", os.environ["PRICING_API_HOST"])

pd.set_option('display.max_columns', None)
from datetime import *


Using as base_host:  http://10.2.142.113:3000/


In [ ]:
df = spark.sql("""
with selected_alternatives as (
  select
    id
    ,created_at as alternative_created_at
    ,fee_experiment_id
    ,name as alternative
  from
    querobolsa_production.fee_experiment_alternatives
  where
    id in (373,374) or (
      fee_experiment_id = (select fee_experiment_id from querobolsa_production.fee_experiment_alternatives where id = 373)
      and name = 'baseline'
    )
),
valid_dates as (
  select
    date
    ,count(fee_experiment_alternatives_history.id)
  from
    data_science.fee_experiment_alternatives_history
    inner join selected_alternatives sa on sa.id = fee_experiment_alternatives_history.id
  where
    ratio > 0
  group by date
  having count(fee_experiment_alternatives_history.id) = (select count(id) from selected_alternatives)
  order by date
)

select
  orders.id as order_id
  ,line_items.offer_id
  ,orders.registered_at
  ,selected_alternatives.fee_experiment_id
  ,selected_alternatives.alternative
  ,base_users.customer_id
  ,orders.checkout_step
  ,orders.price
  ,pre_enrollment_fees.value as pef_value
  ,orders.created_at
  ,case when orders.checkout_step='paid' and coupon_exchanges.id is not null then true else false end exchange
  ,coupons.id as coupon_id
  ,order_origins.origin
from
  querobolsa_production.orders
left join
  querobolsa_production.payments on (orders.id = payments.order_id and payments.status in ('authorized', 'captured'))
left join
  querobolsa_production.coupons on coupons.order_id=orders.id
left join
    querobolsa_production.coupon_exchanges on coupon_exchanges.to_coupon_id=coupons.id
inner join
    querobolsa_production.order_origins on (order_origins.order_id = orders.id)
inner join
  querobolsa_production.base_users on (base_users.id = orders.base_user_id)
inner join
  querobolsa_production.line_items on (line_items.order_id = orders.id)
inner join
  querobolsa_production.pre_enrollment_fees on (pre_enrollment_fees.id = line_items.pre_enrollment_fee_id)
inner join
  querobolsa_production.experiment_pre_enrollment_fees on (experiment_pre_enrollment_fees.pre_enrollment_fee_id = pre_enrollment_fees.id)
inner join
  selected_alternatives on (selected_alternatives.id = experiment_pre_enrollment_fees.fee_experiment_alternative_id)
inner join
  valid_dates on (valid_dates.date = date(orders.registered_at))

where
  orders.registered_at is not null  -- getting only registered orders
  and order_origins.origin = 'Quero Bolsa'  -- excluding OPA, app, etc
  -- and registered_at >= (select max(alternative_created_at) from selected_alternatives)
order by registered_at
""").toPandas()

df['pef_value'] = df['pef_value'].astype(float)
print(df.shape[0])